In [1]:
import pandas as pd
import numpy as np
import requests
import json
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# Configuring the driver to sign in properly so that it can run as fast as possible

opt = webdriver.FirefoxOptions()
opt.add_argument("--headless")
opt.add_argument("--disable-gpu")
opt.add_argument("--disable-dev-shm-usage")
opt.add_argument("--disable-extensions")
opt.add_argument("--disable-browser-side-navigation")
opt.add_argument("--enable-javascript")
opt.add_argument("--disable-infobars")
opt.add_argument("--window-size=1366,768")

In [3]:
# This actually instantiates the driver and opens the sign-in page for Geocaching
driver = webdriver.Firefox(options=opt)

driver.get('https://www.geocaching.com/account/signin?returnUrl=%2fplay')


In [4]:
# Reading in my secrets

with open("secrets.txt") as f:
    lines = f.readlines()
    username = lines[0].strip()
    password = lines[1].strip()
    print(f"USERNAME={username}, PASSWORD={password}")

USERNAME=sspackman99, PASSWORD=Panayaug2018


In [5]:
# locate the username and password fields and enter the login credentials *CREDENTIALS MUST BE PASSED AS STRINGS*
username_field = driver.find_elements(By.ID,"UsernameOrEmail")
password_field = driver.find_elements(By.ID,"Password")

username_field[0].send_keys(str(username))
password_field[0].send_keys(str(password))

In [8]:
# locate the login button and click it
login_button = driver.find_element(By.ID, "SignIn")

login_button.click()

# wait for the login to complete. Will wait until the URL changes
wait = WebDriverWait(driver, 10)
wait.until(EC.url_changes("https://www.geocaching.com/play"))

True

In [9]:
# send a GET request to the API with the logged in cookies
session = requests.Session()
cookies = driver.get_cookies()
for cookie in cookies:
    session.cookies.update({cookie["name"]: cookie["value"]})

api_response = session.get("https://www.geocaching.com/api/proxy/web/search/v2?skip=0&take=50&asc=true&sort=distance&properties=callernote&rad=16093.44&origin=40.2139%2C-111.6336&dorigin=34.03557%2C-84.20132")

In [10]:
# read the response into a pandas df
df = pd.DataFrame(api_response.json())

In [11]:

# close the browser
driver.quit()

In [12]:
# Replace NaN by empty dict
def replace_nans_with_dict(series):
    for idx in series[series.isnull()].index:
        series.at[idx] = {}
    return series



# Explodes list and dicts
def df_explosion(df, col_name:str):

    if df[col_name].isna().any():
        df[col_name] = replace_nans_with_dict(df[col_name])

    df.reset_index(drop=True, inplace=True)

    df1 = pd.DataFrame(df.loc[:,col_name].values.tolist())

    df1.rename(columns={'id': 'attribute.id', 'name': 'attribute.name'}, inplace= True)

    print(df1.columns)
    print(df1.columns[1])

    df = pd.concat([df,df1], axis=1)

    df.drop([col_name], axis=1, inplace=True)

    return df

In [13]:
caches_noExplode = pd.json_normalize(df.results)
caches2explode = caches_noExplode.explode('attributes')

In [14]:
caches = df_explosion(caches2explode, 'attributes')
caches.head()

Index(['attribute.id', 'attribute.name', 'isApplicable'], dtype='object')
attribute.name


,id,name,code,premiumOnly,favoritePoints,geocacheType,containerType,difficulty,terrain,cacheStatus,...,hasCallerNote,distance,bearing,postedCoordinates.latitude,postedCoordinates.longitude,owner.code,owner.username,attribute.id,attribute.name,isApplicable
0,2270967,Game of Thrones - Tyrion Lannister,GC2X0XY,False,1,2,8,2.5,3.5,0,...,False,1555.1mi,W,40.190783,-111.446650,PR25PHM,firennice,NaN,NaN,NaN
1,4568228,Ode To Cold1 - Part 1 - Eat It,GC5C4D4,False,1,5,8,2.0,3.5,0,...,False,1555.2mi,W,40.192383,-111.447267,PR25PHM,firennice,NaN,NaN,NaN
2,893813,Crash - PITAC series,GC1CRWK,False,15,2,3,4.5,4.0,0,...,False,1555.2mi,W,40.185300,-111.449600,PR25PHM,firennice,6.0,Recommended for kids,False
3,893813,Crash - PITAC series,GC1CRWK,False,15,2,3,4.5,4.0,0,...,False,1555.2mi,W,40.185300,-111.449600,PR25PHM,firennice,11.0,May require wading,True
4,2911197,I hate snakes!,GC3JG4G,False,1,5,8,1.5,2.0,0,...,False,1555.7mi,W,40.158550,-111.462700,PR46K6X,stashandcache,1.0,Dogs,True


In [ ]:
caches.pivot(columns='')

In [23]:
pd.json_normalize(df.results).explode('attributes')

,id,name,code,premiumOnly,favoritePoints,geocacheType,containerType,difficulty,terrain,cacheStatus,...,region,country,hasCallerNote,distance,bearing,postedCoordinates.latitude,postedCoordinates.longitude,owner.code,owner.username,attributes
0,2270967,Game of Thrones - Tyrion Lannister,GC2X0XY,False,1,2,8,2.5,3.5,0,...,Utah,United States,False,1555.1mi,W,40.190783,-111.446650,PR25PHM,firennice,NaN
1,4568228,Ode To Cold1 - Part 1 - Eat It,GC5C4D4,False,1,5,8,2.0,3.5,0,...,Utah,United States,False,1555.2mi,W,40.192383,-111.447267,PR25PHM,firennice,NaN
2,893813,Crash - PITAC series,GC1CRWK,False,15,2,3,4.5,4.0,0,...,Utah,United States,False,1555.2mi,W,40.185300,-111.449600,PR25PHM,firennice,"{'id': 6, 'name': 'Recommended for kids', 'isA..."
2,893813,Crash - PITAC series,GC1CRWK,False,15,2,3,4.5,4.0,0,...,Utah,United States,False,1555.2mi,W,40.185300,-111.449600,PR25PHM,firennice,"{'id': 11, 'name': 'May require wading', 'isAp..."
3,2911197,I hate snakes!,GC3JG4G,False,1,5,8,1.5,2.0,0,...,Utah,United States,False,1555.7mi,W,40.158550,-111.462700,PR46K6X,stashandcache,"{'id': 1, 'name': 'Dogs', 'isApplicable': True}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47,7839133,🌞 Sunny Days,GC8XY26,False,0,2,8,2.0,2.0,0,...,Utah,United States,False,1557.8mi,W,40.204052,-111.494482,PR41M20,Jokeerus,"{'id': 18, 'name': 'Dangerous animals', 'isApp..."
47,7839133,🌞 Sunny Days,GC8XY26,False,0,2,8,2.0,2.0,0,...,Utah,United States,False,1557.8mi,W,40.204052,-111.494482,PR41M20,Jokeerus,"{'id': 17, 'name': 'Poisonous plants', 'isAppl..."
48,5347779,Heaven or Las Vegas,GC679JY,False,0,5,2,2.0,2.0,0,...,Utah,United States,False,1557.8mi,W,40.194817,-111.496167,PR9GVEZ,drgw3128,"{'id': 13, 'name': 'Available 24/7', 'isApplic..."
48,5347779,Heaven or Las Vegas,GC679JY,False,0,5,2,2.0,2.0,0,...,Utah,United States,False,1557.8mi,W,40.194817,-111.496167,PR9GVEZ,drgw3128,"{'id': 7, 'name': 'Takes less than one hour', ..."
